In [11]:
import numpy as np # for easier computation
from IPython.display import clear_output # to write new outputs on the previous outputs
import matplotlib.pyplot as plt

In [12]:
def modified_bisection(func, max_search= 10000, min_search=0.0, epsilon=10): # Modified Bisection Search Method
    """
    Returns the root of a given function using Modified Bisection Method. In modified bisection method,
    it is assumed that the function is decreasing, f(max_search) < 0 < f(min_search) and
    it has only one root in [min_search, max_search] interval.

        Parameters:
            func (labmda function): purposed labmda function
            max_search (float): upper bound of the search interval
            min_search (float): lower bound of the search interval (default = 0)
            epsilon (float): tolerance level of the root (default = 0.1)

        Returns:
            mid_point (float): root of the function
    """
    while((max_search - min_search) > epsilon):
        mid_point = (min_search + max_search)/2
        if( func(mid_point) > 0): # Shorten the interval
            min_search = mid_point
        else:
            max_search = mid_point

    return mid_point

In [13]:
def rate_calculator(user_path, W1, W2, sigma): # sinr & rate calculator
    sinr = (np.abs(user_path * W1) ** 2) / (np.abs(user_path * W2) ** 2 + sigma ** 2)
    rate = np.log2(1 + sinr)
    return sinr, rate

In [14]:
def f1_calculator(rate1, rate2, weight1, weight2): # sum-rate calculator
    return weight1 * rate1 + weight2 * rate2

In [15]:
# Define f1a calculator
def f1a_calculator(Rate1, Rate2, sinr1, sinr2, w1, w2, alpha1, alpha2):
    return (w1*Rate1 + w2*Rate2 - w1*alpha1 - w2*alpha2 +
            (w1*(1+alpha1)*sinr1/(1+sinr1)) + (w2*(1+alpha2)*sinr2/(1+sinr2)))

In [16]:
# Define f2a calculator
def f2a_calculator(temp1, temp2, w1, w2, W1, W2, alpha1, alpha2, b1, b2, sigma):
    return (2*np.sqrt(w1*(1+alpha1))*np.real(np.conjugate(b1)*temp1*W1) +
            2*np.sqrt(w2*(1+alpha2))*np.real(np.conjugate(b2)*temp2*W2) -
            (np.abs(b1)**2)*(np.abs(temp1*W1)**2 + np.abs(temp1*W2)**2 + sigma**2) -
            (np.abs(b2)**2)*(np.abs(temp2*W2)**2 + np.abs(temp2*W1)**2 + sigma**2))

In [17]:
# Define f3a calculator
def f3a_calculator(temp1, temp2, w1, w2, W1, W2, alpha1, alpha2, epsilon1, epsilon2, sigma):
    return (2*np.sqrt(w1*(1+alpha1))*np.real(np.conjugate(epsilon1)*temp1*W1) +
            2*np.sqrt(w2*(1+alpha2))*np.real(np.conjugate(epsilon2)*temp2*W2) -
            (np.abs(epsilon1)**2)*(np.abs(temp1*W1)**2 + np.abs(temp1*W2)**2 + sigma**2) -
            (np.abs(epsilon2)**2)*(np.abs(temp2*W2)**2 + np.abs(temp2*W1)**2 + sigma**2))

In [18]:
columns = 10

In [19]:
rate1_no_blockage_with_double = np.zeros((5, columns))
rate2_no_blockage_with_double = np.zeros((5, columns))

In [20]:
for M1 in range(10, 51, 10):
    for column in range(columns):
        # Hyper Parameters
        N  = 20 # Antenna dimension
        # M1 = 30 # IRS1 dimension
        M2 = 20 # IRS2 dimension
        w1 = 1 # First user's weight
        w2 = 1 # Second user's weight
        max_power = 0.01 # Transmitted power
        sigma = 0.0001 # Noise standard deviation
        # Path-Loss exponent
        ALPHA_HIGH = 2.5
        ALPHA_LOW  = 2
        BETWEEN_IRS = 1.8

        # Distances
        BS_IRS1 = 20
        BS_IRS2 = 30
        IRS1_IRS2 = 40
        BS_USER1 = 50
        BS_USER2 = 60
        IRS1_USER1 = 50
        IRS1_USER2 = 40
        IRS2_USER1 = 30
        IRS2_USER2 = 20

        # Channels
        Hs1 = np.matrix(np.sqrt(BS_IRS1**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(M1, N))) # Antenna - IRS1
        Hs2 = np.matrix(np.sqrt(BS_IRS2**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(M2, N))) # Antenna - IRS2

        # Between IRS Signal
        # H12 = np.matrix(np.zeros((M2, M1)))
        H12 = np.matrix(np.sqrt(IRS1_IRS2**-BETWEEN_IRS) * np.exp(2*np.pi*1j*np.random.rand(M2, M1))) # IRS1 - IRS2
        H21 = H12.H # IRS2 - IRS1

        # hsu1 = np.matrix(np.zeros((1, N))) # Blockage Antenna - User1
        hsu1 = np.matrix(np.sqrt(BS_USER1**-ALPHA_HIGH) * np.exp(2*np.pi*1j*np.random.rand(1, N))) # Antenna - User1
        hsu2 = np.matrix(np.sqrt(BS_USER2**-ALPHA_HIGH) * np.exp(2*np.pi*1j*np.random.rand(1, N))) # Antenna - User2

        h1u1 = np.matrix(np.sqrt(IRS1_USER1**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(1, M1))) # IRS1 - User1
        h1u2 = np.matrix(np.sqrt(IRS1_USER2**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(1, M1))) # IRS1 - User2

        # h2u1 = np.matrix(np.zeros((1,M2))) # Blockage IRS2 - User1
        h2u1 = np.matrix(np.sqrt(IRS2_USER1**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(1, M2))) # IRS2 - User1
        h2u2 = np.matrix(np.sqrt(IRS2_USER2**-ALPHA_LOW) * np.exp(2*np.pi*1j*np.random.rand(1, M2))) # IRS2 - User2

        # Passive Beamforming Coefficients Initialization
        theta1 = np.exp(1j * 2*np.pi * np.random.rand(1, M1)[0])
        theta2 = np.exp(1j * 2*np.pi * np.random.rand(1, M2)[0])
        phi1 = np.matrix(np.diag(theta1))
        phi2 = np.matrix(np.diag(theta2))
        theta1_temp = theta1
        theta2_temp = theta2

        # Path for each user
        user1_path = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
        user2_path = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1

        # Active Beamforming Coefficients Initialization
        W1 = np.random.rand(N, 1)
        W2 = np.random.rand(N, 1)
        W1_temp = W1
        W2_temp = W2

        gamma = 0.1

        # Coordinate Descent
        mini_result = []
        i = 0
        # while True:
        while((i <= 50) or ((mini_result[-1] - mini_result[-50]) > 0.01)):
            i += 1
            # -------------------------------------------------------------------------------------------------------------------------------
            # To remove log2
            for _ in range(1):
                sinr1, rate1 = rate_calculator(user1_path, W1, W2, sigma)
                sinr2, rate2 = rate_calculator(user2_path, W2, W1, sigma)
                alpha1 = sinr1
                alpha2 = sinr2
            # -------------------------------------------------------------------------------------------------------------------------------
            # Active Beamforming Optimization
            for _ in range(1):
                # Fraction to Linear
                b1 = (np.sqrt(w1 * (1 + alpha1)) * user1_path * W1) / (np.abs(user1_path * W1)**2 + np.abs(user1_path * W2)**2 + sigma**2)
                b2 = (np.sqrt(w2 * (1 + alpha2)) * user2_path * W2) / (np.abs(user2_path * W2)**2 + np.abs(user2_path * W1)**2 + sigma**2)

                # Lagrangian dual transformation
                temp_func = lambda lambda_temp : (np.linalg.norm(complex(np.sqrt(w1 * (1 + alpha1)) * b1) * np.linalg.inv(lambda_temp * np.eye(N, dtype= "complex128") +
                            float(np.abs(b1))**2 * user1_path.getH() * user1_path + float(np.abs(b2))**2 *user2_path.getH() * user2_path) *
                            user1_path.getH())**2 + np.linalg.norm(complex(np.sqrt(w2 * (1 + alpha2)) * b2) * np.linalg.inv(lambda_temp * np.eye(N, dtype= "complex128") +
                            float(np.abs(b1))**2 * user1_path.getH() * user1_path + float(np.abs(b2))**2 * user2_path.getH() * user2_path) *
                            user2_path.getH())**2 - max_power)

                # Lagrangian dual variable
                lambda_var = modified_bisection(temp_func, max_search=max_search)
                
                # Antenna beamforming(Active beamforming)
                W1_before = W1
                W2_before = W2

                W1 = (complex(np.sqrt(w1 * (1 + alpha1)) * b1) * np.linalg.inv(lambda_var * np.eye(N, dtype= "complex128") + float(np.abs(b1))**2 * user1_path.getH() *
                    user1_path + float(np.abs(b2))**2 * user2_path.getH() * user2_path) * user1_path.getH())
                W2 = (complex(np.sqrt(w2 * (1 + alpha2)) * b2) * np.linalg.inv(lambda_var * np.eye(N, dtype= "complex128") + float(np.abs(b1))**2 * user1_path.getH() *
                    user1_path + float(np.abs(b2))**2 * user2_path.getH() * user2_path) * user2_path.getH())
            # -------------------------------------------------------------------------------------------------------------------------------
            for _ in range(1):
                # Fraction to Linear
                epsilon1 = (np.sqrt(w1*(1+alpha1)) * user1_path*W1) / (np.abs(user1_path*W1)**2 + np.abs(user1_path*W2)**2 + sigma**2)
                epsilon2 = (np.sqrt(w2*(1+alpha2)) * user2_path*W2) / (np.abs(user2_path*W2)**2 + np.abs(user2_path*W1)**2 + sigma**2)

                # IRS1 optimization
                a00 = (np.matrix(np.diag(np.array(h1u1)[0]))*Hs1 +
                                    np.matrix(np.diag(np.array(h1u1)[0]))*H21*phi2*Hs2 +
                                    np.matrix(np.diag(np.array(h2u1*phi2*H12)[0]))*Hs1)*W1
                a10 = (np.matrix(np.diag(np.array(h1u1)[0]))*Hs1 +
                                    np.matrix(np.diag(np.array(h1u1)[0]))*H21*phi2*Hs2 +
                                    np.matrix(np.diag(np.array(h2u1*phi2*H12)[0]))*Hs1)*W2
                a01 = (np.matrix(np.diag(np.array(h1u2)[0]))*Hs1 +
                                    np.matrix(np.diag(np.array(h1u2)[0]))*H21*phi2*Hs2 +
                                    np.matrix(np.diag(np.array(h2u2*phi2*H12)[0]))*Hs1)*W1
                a11 = (np.matrix(np.diag(np.array(h1u2)[0]))*Hs1 +
                                    np.matrix(np.diag(np.array(h1u2)[0]))*H21*phi2*Hs2 +
                                    np.matrix(np.diag(np.array(h2u2*phi2*H12)[0]))*Hs1)*W2
                b = np.matrix([[((hsu1 + h2u1*phi2*Hs2)*W1)[0,0],
                                ((hsu2 + h2u2*phi2*Hs2)*W1)[0,0]],
                                [((hsu1 + h2u1*phi2*Hs2)*W2)[0,0],
                                ((hsu2 + h2u2*phi2*Hs2)*W2)[0,0]]])
                U = (np.abs(epsilon1)**2)[0,0] * (a00*(a00.H) + a10*(a10.H)) + \
                    (np.abs(epsilon2)**2)[0,0] * (a01*(a01.H) + a11*(a11.H))
                v = (np.sqrt(w1*(1+alpha1))[0,0] * np.conjugate(epsilon1)[0,0] * a00 -
                        (np.abs(epsilon1)[0,0]**2) * (np.conjugate(b[0,0]) * a00 + np.conjugate(b[1,0]) * a10)) + \
                    (np.sqrt(w2*(1+alpha2))[0,0] * np.conjugate(epsilon2)[0,0] * a11 -
                        (np.abs(epsilon2)[0,0]**2) * (np.conjugate(b[0,1]) * a01 + np.conjugate(b[1,1]) * a11))

                first_part = np.array((-U * np.matrix(theta1).transpose().conjugate() + v).conjugate())
                second_part = np.array((-1j * np.matrix(theta1).transpose().conjugate()))
                derivative = 2 * np.real(first_part * second_part).transpose()[0]
                phase = np.angle(theta1) + gamma*derivative
                theta1 = np.exp(1j * phase)
                # ------------------------------- #
                phi1 = np.matrix(np.diag(theta1))

                # IRS2 optimization
                a00 = (np.matrix(np.diag(np.array(h2u1)[0]))*Hs2 +
                                    np.matrix(np.diag(np.array(h1u1*phi1*H21)[0]))*Hs2 +
                                    np.matrix(np.diag(np.array(h2u1)[0]))*H12*phi1*Hs1)*W1
                a10 = (np.matrix(np.diag(np.array(h2u1)[0]))*Hs2 +
                                    np.matrix(np.diag(np.array(h1u1*phi1*H21)[0]))*Hs2 +
                                    np.matrix(np.diag(np.array(h2u1)[0]))*H12*phi1*Hs1)*W2
                a01 = (np.matrix(np.diag(np.array(h2u2)[0]))*Hs2 +
                                    np.matrix(np.diag(np.array(h1u2*phi1*H21)[0]))*Hs2 +
                                    np.matrix(np.diag(np.array(h2u2)[0]))*H12*phi1*Hs1)*W1
                a11 = (np.matrix(np.diag(np.array(h2u2)[0]))*Hs2 +
                                    np.matrix(np.diag(np.array(h1u2*phi1*H21)[0]))*Hs2 +
                                    np.matrix(np.diag(np.array(h2u2)[0]))*H12*phi1*Hs1)*W2
                b = np.matrix([[((hsu1 + h1u1*phi1*Hs1)*W1)[0,0],
                                ((hsu2 + h1u2*phi1*Hs1)*W1)[0,0]],
                                [((hsu1 + h1u1*phi1*Hs1)*W2)[0,0],
                                ((hsu2 + h1u2*phi1*Hs1)*W2)[0,0]]])
                U = (np.abs(epsilon1)**2)[0,0] * (a00*(a00.H) + a10*(a10.H)) + \
                    (np.abs(epsilon2)**2)[0,0] * (a01*(a01.H) + a11*(a11.H))
                v = (np.sqrt(w1*(1+alpha1))[0,0] * np.conjugate(epsilon1)[0,0] * a00 -
                        (np.abs(epsilon1)[0,0]**2) * (np.conjugate(b[0,0]) * a00 + np.conjugate(b[1,0]) * a10)) + \
                    (np.sqrt(w2*(1+alpha2))[0,0] * np.conjugate(epsilon2)[0,0] * a11 -
                        (np.abs(epsilon2)[0,0]**2) * (np.conjugate(b[0,1]) * a01 + np.conjugate(b[1,1]) * a11))

                first_part = np.array((-U * np.matrix(theta2).transpose().conjugate() + v).conjugate())
                second_part = np.array((-1j * np.matrix(theta2).transpose().conjugate()))
                derivative = 2 * np.real(first_part * second_part).transpose()[0]
                phase = np.angle(theta2) + gamma*derivative
                theta2 = np.exp(1j * phase)
                # ------------------------------- #
                phi2 = np.matrix(np.diag(theta2))

                user1_path = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
                user2_path = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1
            # -------------------------------------------------------------------------------------------------------------------------------

                sinr1, rate1 = rate_calculator(user1_path, W1, W2, sigma)
                sinr2, rate2 = rate_calculator(user2_path, W2, W1, sigma)

                mini_result.append(f1_calculator(rate1, rate2, w1, w2)[0, 0])
                
                if i % 5 == 0:
                    print("Sample:", columns * (M1/10 - 1) + (column + 1))
                    print("Lambda Variable: ", lambda_var)
                    print("Ratio Power in percent:", 100 * np.abs(np.linalg.norm(W1)**2 + np.linalg.norm(W2)**2 - max_power)/max_power)
                    print("Rate1 : ", rate1[0, 0])
                    print("Rate2 : ", rate2[0, 0])
                    print("Sum-Rate :", f1_calculator(rate1, rate2, w1, w2)[0, 0])
                    # print("f3a-Value :", f3a_calculator(user1_path, user2_path, w1, w2, W1, W2, alpha1, alpha2, epsilon1, epsilon2, sigma)[0, 0])
                    print("Theta1 phase: ", np.angle(theta1) * 180 / np.pi)
                    print("Theta2 phase: ", np.angle(theta2) * 180 / np.pi)
                    print("Gamma : ", gamma)
                    clear_output(wait=True)
                    
        # H12 = np.matrix(np.sqrt(IRS1_IRS2**-BETWEEN_IRS) * np.exp(2*np.pi*1j*np.random.rand(M2, M1))) # IRS1 - IRS2
        # H21 = H12.H # IRS2 - IRS1
        # user1_path = hsu1 + h1u1*phi1*Hs1 + h2u1*phi2*Hs2 + h1u1*phi1*H21*phi2*Hs2 + h2u1*phi2*H12*phi1*Hs1
        # user2_path = hsu2 + h1u2*phi1*Hs1 + h2u2*phi2*Hs2 + h1u2*phi1*H21*phi2*Hs2 + h2u2*phi2*H12*phi1*Hs1
        # sinr1, rate1 = rate_calculator(user1_path, W1, W2, sigma)
        # sinr2, rate2 = rate_calculator(user2_path, W2, W1, sigma)
        rate1_no_blockage_with_double[int(M1/10)-1, column] = rate1[0, 0]
        rate2_no_blockage_with_double[int(M1/10)-1, column] = rate2[0, 0]

Sample: 50.0
Lambda Variable:  494.76511776447296
Ratio Power in percent: 0.002981059929966362
Rate1 :  11.488881233023525
Rate2 :  9.750599304247991
Sum-Rate : 21.23948053727152
Theta1 phase:  [ 139.06872976   79.76044996  158.7728195   -83.4198372  -142.57832238
 -106.29638326   68.3676831    -8.882888      6.70647254 -127.18999572
  -61.11214428   29.10192128  -56.37334727  -74.7868089   -42.56043502
 -123.31815601  118.05629858   76.96579168  -69.78033017   -3.91384313
 -135.26659545  169.46889365  -32.37911201  -51.14595063   13.53970903
  152.04402721  133.91178282   -7.19868291   47.99697534  -77.62543368
  -45.82757498  111.62466799 -147.00857387   71.99704084   42.18354948
  127.74047496 -177.10253021   51.79486121   61.18710745  114.61693618
  -47.20354149  -96.64823928 -137.52213767  148.74511036  -62.92697503
  -70.47014071   19.1345206  -122.68660868   43.96845291  160.76278075]
Theta2 phase:  [ 1.99978191e+00  1.46827584e-01  1.08404482e+02  4.81233547e+01
 -5.26025185e+0

In [21]:
with open('rate1_no_blockage_with_double.npy', 'wb') as f:
    np.save(f, rate1_no_blockage_with_double)

In [22]:
with open('rate2_no_blockage_with_double.npy', 'wb') as f:
    np.save(f, rate2_no_blockage_with_double)

In [23]:
rate1_no_blockage_with_double[:, 0]

array([10.78534659, 11.16384688, 10.12833066, 10.5840142 , 11.34938391])

In [24]:
rate2_no_blockage_with_double[:, 0]

array([11.33257776, 11.66301715,  8.14108568, 10.22045744,  9.53896816])